# Convolutional Neural Network

### Importing the libraries

In [16]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
# Prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale= 1./255,         # Feature scaling
    shear_range= 0.2,        
    zoom_range= 0.2,         
    horizontal_flip= True    
)

training_set = train_datagen.flow_from_directory(
    'CNN_dataset/training_set',
    target_size= (64, 64),    # Low image size to put less load on the machine
    batch_size= 32,           # Default value
    class_mode= 'binary'      # Only 2 outcomes (cat/dog), so binary. Use 'categorical' for multi-class classification
)


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255) # applying same feature scalling

test_set = test_datagen.flow_from_directory(
        'CNN_dataset/test_set',
        target_size= (64, 64),
        batch_size= 32,
        class_mode= 'binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
from tensorflow.keras.models import Sequential

cnn = Sequential()

### Step 1 - Convolution

In [6]:
from tensorflow.keras.layers import Conv2D

cnn.add(Conv2D(
    filters= 32, 
    kernel_size= 3, 
    activation= 'relu', 
    input_shape= [64, 64, 3]
))

# input_shape=[64, 64, 3] assumes RGB images. 
# If the images were in black & white, use input_shape=[64, 64, 1]


c:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [7]:
from tensorflow.keras.layers import MaxPooling2D

cnn.add(MaxPooling2D(
    pool_size= 2,        # Size of the pooling window (height, width)
    strides= 2           # Stride of the pooling operation
))


### Adding a second convolutional layer

In [8]:
cnn.add(Conv2D(
    filters=32, 
    kernel_size=3, 
    activation='relu'   #input shape not required as it was fixed in the 1st layer. so it will get the same size as input
))

MaxPooling2D(
    pool_size= 2,
    strides= 2
)

<MaxPooling2D name=max_pooling2d_1, built=False>

### Step 3 - Flattening

In [9]:
from tensorflow.keras.layers import Flatten

cnn.add(Flatten())


### Step 4 - Full Connection

In [10]:
from tensorflow.keras.layers import Dense

cnn.add( Dense(
    units = 128,
    activation= 'relu'
))


### Step 5 - Output Layer

In [11]:
cnn.add( Dense(
    units = 1,                  # binary output so only one layer required
    activation= 'sigmoid'       # sigmoid for output layer
))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data= test_set, epochs=25)

Epoch 1/25


c:\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 38s 144ms/step - accuracy: 0.5626 - loss: 0.6876 - val_accuracy: 0.6360 - val_loss: 0.6376
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.6839 - loss: 0.5973 - val_accuracy: 0.6430 - val_loss: 0.6388
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.6997 - loss: 0.5733 - val_accuracy: 0.6785 - val_loss: 0.6107
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7127 - loss: 0.5551 - val_accuracy: 0.7365 - val_loss: 0.5378
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7380 - loss: 0.5254 - val_accuracy: 0.7480 - val_loss: 0.5216
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7493 - loss: 0.5078 - val_accuracy: 0.7605 - val_loss: 0.5165
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7683 - loss: 0.4823 - val_accuracy: 0.7455 - val_loss: 0.5199
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.7654 - loss: 0.4732 - val_accura

### Saving model

In [20]:
cnn.save('CNN_model/cnn_cat_or_dog_v1.0.keras')

## Part 4 - Making a single prediction

In [39]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
cnn = tf.keras.models.load_model('CNN_model/cnn_cat_or_dog_v1.0.keras')

test_image = image.load_img('CNN_dataset/single_prediction/cat_or_dog_10.jpg', target_size =(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis= 0)
result = cnn.predict(test_image)
print(training_set.class_indices)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
{'cats': 0, 'dogs': 1}


In [40]:
print(prediction)

dog
